In [1]:
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import json
import requests
import pandas as pd
import io
import geopy.distance

def arrondissements_geojson():
    url = "https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr"
    response = requests.get(url, verify=False)
    return response.json()

def arrondissements_dataframe():
    url = "https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=csv&timezone=Europe/Berlin&lang=fr"
    response = requests.get(url, verify=False).content
    df = pd.read_csv(io.StringIO(response.decode('utf-8')), sep=";")
    df = df[['c_ar', 'l_aroff', 'l_ar']].rename(columns={"c_ar": "numero", "l_ar": "nom_1", "l_aroff": "nom_2"}).sort_values("numero").reset_index(drop=True)
    df['Information'] = df['nom_1'].str.replace("Ardt", "Arrondissement") + " (" + df['nom_2'] + ")"
    return df

def metro_dataframe():
    url = "https://data.iledefrance-mobilites.fr/explore/dataset/arrets-lignes/download/?format=csv&timezone=Europe/Berlin&lang=fr"
    response = requests.get(url, verify=False).content
    df = pd.read_csv(io.StringIO(response.decode('utf-8')), sep=";")
    
    # Renommer les modes de transport en français
    mode_translation = {
        'Metro': 'Métro',
        'regionalRail': 'TER',
        'LocalTrain': 'Transilien',
        'Tramway': 'Tramway',
        'Bus': 'Bus',
        'RailShuttle': 'Navette ferroviaire',
        'RapidTransit': 'RER',
        'Funicular': 'Funiculaire'
    }
    
    df['mode'] = df['mode'].map(mode_translation)
    df['nom'] = df['mode'] + " " + df['route_long_name']
    df['info'] = df['nom'] + ' - ' + df['stop_name']
    return df

def arrondissements_map():
    fig = px.choropleth_mapbox(arrondissements_dataframe(), geojson=arrondissements_geojson(),
                               locations="numero", featureidkey="properties.c_ar",
                               color="Information", hover_name="Information",
                               mapbox_style="open-street-map", zoom=11, center={"lat": 48.8566, "lon": 2.3522}, opacity=0.5,
                               hover_data={col : False for col in arrondissements_dataframe().columns})
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, showlegend=True)
    return fig


def stations_map(stations: pd.DataFrame):
    fig = px.scatter_mapbox(stations, lat="stop_lat", lon="stop_lon", color="nom", hover_name="info", hover_data={col: False for col in stations.columns},
                            mapbox_style="open-street-map", zoom=11, center={"lat": 48.8566, "lon": 2.3522})
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, showlegend=True)
    fig.update_traces(marker=dict(size=10))
    return fig

def complete_stations_map(stations: pd.DataFrame):
  fig = arrondissements_map()
  fig_stations = stations_map(stations)
  for trace in fig_stations.data:
      fig.add_trace(trace)
  return fig

def stations_distance(station_1_lat, station_1_lon, station_2_lat, station_2_lon):
    return geopy.distance.geodesic((station_1_lat, station_1_lon), (station_2_lat, station_2_lon)).km

In [25]:
# We retrieve all the stations in Paris
stations = metro_dataframe()

# We find all the stations linked to "La Défense"
defense_stations = stations[stations['stop_name'].str.contains("Défense")]['nom'].unique()

# We keep those way of transport
stations = stations[stations['nom'].isin(defense_stations)]

# # We find all the metro stations being close to at least metro 1 stations
# metro_1_stations = stations[stations['nom'] == "Métro 1"].copy(deep=True)[['nom', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
# metro_other_stations = stations[(stations['mode'] == "Métro") & (stations['route_long_name'] != "1")].copy(deep=True)[['nom', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
# metro_distances = metro_other_stations.merge(metro_1_stations, how="cross", suffixes=("_other", "_1"))
# metro_distances['distance'] = metro_distances.apply(lambda row: stations_distance(row['stop_lat_other'], row['stop_lon_other'], row['stop_lat_1'], row['stop_lon_1']), axis=1)
# metro_distances = metro_distances[metro_distances['distance'] <= 0.2]

# # metro_stations_distances = metro_stations.copy(deep=True).merge(metro_stations.copy(deep=True), how="cross", suffixes=("_1", "_2"))
# # metro_stations_distances['distance'] = metro_stations_distances.progress_apply(lambda row : stations_distance(row['stop_lat_1'], row['stop_lon_1'], row['stop_lat_2'], row['stop_lon_2']), axis=1)
# # metro_stations_distances

C:\Users\DLEBDGNL\Documents\Private\paris_map\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.iledefrance-mobilites.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [26]:
fig = complete_stations_map(stations)
fig.write_html(r"./paris_map.html")

C:\Users\DLEBDGNL\Documents\Private\paris_map\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.paris.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\DLEBDGNL\Documents\Private\paris_map\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.paris.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\DLEBDGNL\Documents\Private\paris_map\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.paris.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-